In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import re
import liikutils as lu
from scipy import stats


In [2]:
lu.load_data()

perus = pd.DataFrame(lu.perus_df.rename(columns=
                                {'SES-indeksi':'ses',
                                 'Opiskelijamäärä yhteensä':'n_opp',
                                 'Liikunnan painotettu opetus (1=kyllä, tyhjä=ei)':'painot'}).set_index('Koulu_ID'),
             columns=['ses','painot','n_opp'])

perus.painot = perus.painot.fillna(0)
#len(lu.perus_df.Koulu_ID.unique()), len(lu.perus_df)
perus.shape

(102, 3)

In [3]:
def hae_data(nimi : str, ryhma : str = '', vuosi = 2023):
    '''
    Hakee datat. Nimi voi olla
        - ktk
        - nyky
        - piha
        - sali
        - pyora
    '''
    df = None
    if (nimi == 'ktk'):
        if (ryhma==''):
            return pd.concat([hae_data('ktk',x,vuosi) for x in ['5t','5p','8t','8p']])
        df = pd.read_csv("../data/ktk_%d_%s_g.tsv"%(vuosi,ryhma),sep="\t")
            
    if (nimi == 'nyky'):
        df = lu.nykytila_df[lu.nykytila_df.Vastaamisajankohta.isin(['%d kevät'%vuosi,'%d syksy'%(vuosi-1)])].\
            drop_duplicates(['Koulu ID']).rename(columns={'Koulu ID':'Koulu_ID'})
        if ryhma=='':
            df.columns = ["k_"+re.sub(" .*","",k).replace(".","_") if k[1]=='.' else k for k in list(df.columns)]
        df.set_index('Koulu_ID',inplace=True)
        df = pd.DataFrame(df,columns=df.columns[5:])

            
    if (nimi == 'piha'):
        df = pd.read_excel("../input/Pihojen_kuntoarviot_%s_suojattu.xlsx"%(vuosi+1),header=1)
        df.columns=['pihakunto','Koulu_ID','stara_kommentti']
        df.set_index('Koulu_ID',inplace=True)
        df.pihakunto = pd.to_numeric(df.pihakunto, errors='coerce').fillna(-1)
        df = df.sort_values('pihakunto').reset_index().drop_duplicates(['Koulu_ID'],keep='last').set_index('Koulu_ID')

        
        
    if (nimi == 'sali'):
        df = pd.DataFrame(pd.read_excel("../input/Peruskoulujen_liikuntasalit_suojattu.xlsx").\
                          set_index('Koulu_ID').sum(axis=1),
                          columns=['salim2'])
        df = df.groupby('Koulu_ID').mean()

        
    if (nimi == 'pyora'):
        p = pd.DataFrame(
                pd.read_excel("../input/Koulut_pyorapysakointipaikkojen_maarat_suojattu.xlsx").\
                    set_index('Koulu_ID').rename(columns={'Pyöräpysäköintipaikkojen lukumäärä':'pyorapk'}),
                columns=['pyorapk']).fillna(0).astype(int).merge(perus,on='Koulu_ID',how='right')
        p['pk_opp']=(p.pyorapk/p.n_opp).round(2)
        df = pd.DataFrame(zip(p.pyorapk,p.pk_opp),index=p.index,columns=['paikat','pk_opp']).dropna()
        df = df.groupby('Koulu_ID').mean().round(2)

        
    if (nimi == 'liitu'):
        liitu_ymp = pd.read_csv("../data/liitu_kouluymp1.csv").set_index('Koulu_ID')
        df = liitu_ymp.rename(columns={'En osaa sanoa':'EOS','Kyllä':'vast_K','Ei':'vast_E'})
        

        
    return df
    

## Datojen mallit

### Liitu kouluympäristö1 kannustajat

In [4]:
liitu = hae_data('liitu')
liitu.sample(5)

,Muuttuja,vast_E,EOS,vast_K,vuosi
Koulu_ID,,,,,
115,Jokaiseen koulupäivään sisältyy vähintään 30 m...,45.8,0.0,54.2,2022
119,Välituntivälineet ovat kaikkien saatavilla,31.2,31.2,37.5,2022
124,Koulun pihalla on pelialue ja riittävästi muit...,4.2,25.0,70.8,2022
140,Koulun liikuntasali on käytössä välituntiliikk...,100.0,0.0,0.0,2022
124,Välituntivälineet ovat kaikkien saatavilla,10.4,27.1,62.5,2022


### Pihakuntoarvio

In [5]:
piha = hae_data('piha')
piha.sample(5)

,pihakunto,stara_kommentti
Koulu_ID,,
127,2.0,Peliareena ja asfalttimaalaukset tullut 2016. ...
189,3.0,Valmistunut 2019
103,4.0,Koulupihan välineet huonossa kunnossa ja toimi...
125,4.0,Perusparannus tulossa.
120,3.0,Piha uusittu 2020.


In [6]:
print(piha.sample(4))
kunnot = piha['pihakunto'].astype(str).value_counts().sort_index()
#kunnot.to_excel('../output/piha_kunnot_2023.xlsx')
kunnot

          pihakunto                                    stara_kommentti
Koulu_ID                                                              
111             3.0  Pihaa uusittu 2020. Puiston puolen piha kaipai...
198             3.0                                 Piha uusittu 2019.
145             4.0  Vanhimmat välineet vuodelta 1998 ja loput 2006...
171             4.0                             Välineet alkuperäisiä.


pihakunto
1.0    10
2.0    22
3.0    42
4.0    20
Name: count, dtype: int64

### Salien koot ja salitila per oppilas

In [7]:
sali = hae_data('sali')


In [8]:
print(sali.sample(5))
sali_opp=sali.merge(perus,on='Koulu_ID',how='left')
sali_opp['m2opp']=sali_opp.salim2/sali_opp.n_opp
sali_oppm2 = ((sali_opp['m2opp']*10.0).round())/10
opp_salitila = sali_oppm2.value_counts().sort_index()
# opp_salitila.to_excel('../output/opp_salitila_2023.xlsx')
opp_salitila



          salim2
Koulu_ID        
147        398.0
105        231.5
161        214.0
178        171.0
110        800.0


m2opp
0.1      1
0.2      2
0.3      7
0.4     11
0.5     13
0.6      9
0.7      9
0.8      6
0.9      5
1.0      2
1.1      5
1.2      6
1.3      3
1.5      2
1.6      2
1.7      1
1.8      1
2.3      1
2.4      1
11.9     1
Name: count, dtype: int64

In [9]:
salikoot = ((hae_data('sali')/50).round()*50).value_counts().sort_index().reindex(fill_value=np.arange(0,500,50))
# salikoot.to_excel('../output/salikoot_2023.xlsx')
salikoot

salim2
100.0      3
150.0      9
200.0     13
250.0     16
300.0      9
350.0      8
400.0     16
450.0      5
500.0      3
550.0      2
600.0      2
650.0      1
800.0      1
Name: count, dtype: int64

### Pyöräpaikat

In [10]:
pyora = hae_data('pyora')
pyora.sample(4)

,paikat,pk_opp
Koulu_ID,,
106,50.0,0.10
179,0.0,0.00
188,45.0,0.12
198,70.0,0.53


In [11]:
pk = pyora.merge(perus,on='Koulu_ID',how='left')
print(pk.sample(3))
# pk.to_excel("../output/pyorapaikat_2023.xlsx") # pyorapaikat, sitten per oppilas jakauma

pk_opp = pk.pk_opp.round(1).value_counts().sort_index()
print(pk_opp)
# pk_opp.to_excel("../output/opp_pyorapaikat_2023.xlsx")


          paikat  pk_opp   ses  painot  n_opp
Koulu_ID                                     
201         30.0    0.08  98.0     0.0    398
128         59.0    0.07  95.0     0.0    886
134         39.0    0.08  68.0     0.0    461
pk_opp
0.0    16
0.1    39
0.2    13
0.3    12
0.4     5
0.5     6
0.6     3
0.7     2
0.9     1
1.1     2
Name: count, dtype: int64


### KTK Kouluterveyskysely, liikkuvuusryhmät

In [12]:

ktk = hae_data('ktk','',2023)
ktk

,Koulu_ID,vuosi,g_ex_012d,g_ex_34d,g_ex_56d,g_ex_7d,sukupuoli,luokka
0,101,2023,14.8,18.5,48.1,18.5,t,5
1,102,2023,2.3,16.3,44.2,37.2,t,5
2,103,2023,7.6,15.0,31.2,46.2,t,5
3,104,2023,2.0,4.1,24.5,69.4,t,5
4,106,2023,16.4,14.9,31.3,37.3,t,5
...,...,...,...,...,...,...,...,...
37,189,2023,9.5,26.0,30.1,34.2,p,8
38,193,2023,7.2,14.3,17.8,60.7,p,8
39,194,2023,11.8,30.8,32.3,25.0,p,8
40,200,2023,100.0,0.0,0.0,0.0,p,8


In [13]:
ktk_os_ryhmittain=pd.DataFrame(
    ktk.groupby(['luokka','sukupuoli']).g_ex_012d.mean().round(2)
)
ktk_os_ryhmittain['g_ex_7d']=pd.DataFrame(
    ktk.groupby(['luokka','sukupuoli']).g_ex_7d.mean().round(2)
)

ktk_os_ryhmittain['N']=pd.DataFrame(ktk.groupby(['luokka','sukupuoli']).luokka.value_counts())

# ktk_os_ryhmittain.to_excel("../output/ktk_os_ryhmittain_2023.xlsx")

ktk_os_ryhmittain

g_ex_012d  g_ex_7d   N
luokka sukupuoli                        
5      p               6.59    45.16  90
       t               6.91    37.89  90
8      p              13.54    30.83  42
       t              23.02    23.77  41

### Nykytila
- _NNNN_top: Nykytilan arvion eniten pisteitä saaneet toimenpiteet
- # Nykytila_%d_kys_osuudet: Nykytilan arvion kysymysten vastausten jakaumat

In [14]:
nyky_k = hae_data('nyky','k')
nyky = hae_data('nyky')

In [15]:

nykypist = pd.DataFrame(zip(np.sum(np.array(nyky_k)[:,:-1],axis=0),nyky_k.columns)).sort_values(0)

nykypist #.to_excel("../output/nykytila_2023_top.xlsx")


,0,1
18,114,3.7 Koulun henkilökunnan kanssa keskustellaan ...
26,119,4.7 Koulun liikuntasali on käytössä välituntil...
33,127,6.3 Kouluterveydenhoitaja on mukana oppilaiden...
32,127,6.2 Huoltajien kanssa tehdään yhteistyötä käve...
10,130,"2.4 Koulun opetustiloja on muokattu siten, ett..."
22,133,4.3 Oppilaat toimivat välituntitoimintaan akti...
16,139,3.5 Koululla on kohdennettuja toimenpiteitä vä...
15,150,3.4 Oppilaat ovat järjestämässä Liikkuva koulu...
34,153,6.4 Koulu tekee yhteistyötä kunnan eri hallint...
6,153,1.7 Opettajia osallistuu liikunnallisiin tai t...


In [16]:
nyky_k = hae_data('nyky','k',2023)
nyky_kys_os = pd.DataFrame(nyky_k,columns = nyky_k.columns[0:-1]).apply(pd.Series.value_counts).T 
nyky_kys_os.sample(4)
# ...  .to_excel("../output/nykytila_2023_kys_osuudet.xlsx")

,0,1,2,3,4
6.1 Opettajat tekevät keskenään yhteistyötä yli oppiainerajojen aktiivisuuden lisäämiseksi.,3.0,8.0,20.0,28.0,15.0
"1.5 Liikkuva koulu -toiminta on kirjattu kuntastrategiaan, kunnan hyvinvointistrategiaan tai muuhun vastaavaan asiakirjaan.",4.0,3.0,15.0,24.0,28.0
3.2 Oppilaat ovat mukana suunnittelemassa ja järjestämässä koko koulun yhteisiä liikunnallisia tapahtumia.,1.0,9.0,16.0,34.0,14.0
4.7 Koulun liikuntasali on käytössä välituntiliikkumisessa.,24.0,15.0,12.0,12.0,11.0


# Korrelaatio-osuus

In [17]:
def korr_ajo(df : pd.DataFrame, f1,f2):
    '''
    Kahden muuttujan korrelaatioanalyysi
    '''
    
    c_df = pd.DataFrame(df,columns=[f1,f2]).dropna()#.drop_duplicates()
    if len(c_df)<2:
        pearson = (0,1)
    else:
        pearson = stats.pearsonr(c_df[f1],c_df[f2])
    
    details = {'N':len(c_df), 'f1':f1, 'f2':f2}
    if ('luokka' in df.columns):
        details['luokka']=list(df.luokka.unique())
    if ('sukupuoli' in df.columns):
        details['sukupuoli']=list(df.sukupuoli.unique())
    if ('vuosi' in df.columns):
        details['vuosi']=list(df.vuosi.unique())
            

    return pearson, details

### Korrelaation laskenta ...

In [18]:
ktk = hae_data('ktk','',2023)
nyky = hae_data('nyky','',2023)
liitu = hae_data('liitu','',2023)

piha = hae_data('piha')
sali = hae_data('sali')
pyora = hae_data('pyora')


movedf = pd.read_csv("../data/move-osuudet_2022.tsv",sep="\t")
move = pd.DataFrame(movedf,columns=['luokka','sukupuoli','osuus_A','osuus_C','Koulu_ID']).set_index('Koulu_ID')

# ktk_nyky = ktk.merge(nyky,on='Koulu_ID')
# ktk_nyky.to_excel("../output/ktk_nyky_2023.xlsx")



In [19]:
#korr_ajo(ktk_nyky,"g_ex_012d","k_2_4") ,\
#korr_ajo(olo_muut[olo_muut.luokka==5],"g_ex_012d","pihakunto")

### Korrelaatiotestit peräkkäin:

In [20]:
def korr_testit(df,vasteet,faktorit,osajoukot,kynnys=0.05):
    res = []
    for oj in osajoukot:
        o_dfs = [('yht',df)] if (len (oj) == 0) else [(g,gdf) for (g,gdf) in df.groupby(oj)]
        for g,gdf in o_dfs:
            print ("================= Ryhmä %s N=%d ===============" %(g,len(gdf.drop_duplicates())))
            for vaste in vasteet:
                for f in faktorit:
                    #print ("Vaste: %s" %(vaste))
                        (p,d) = korr_ajo(gdf,vaste,f)
                        if (p[1]<=kynnys):
                            print("%20s x %-20s Pearson r %+3.3f p=%1.3f N=%d %s"%(vaste,f[0:20],p[0],p[1],d['N'],'*' * int(np.log(.05)-np.log(p[1]+.0001))))
                            res.append((str(g),vaste,f,p[0],p[1],d['N']))
    return pd.DataFrame(res,columns=['osajoukko','vaste','tekija','r','p_arvo','otos_N'])
                            
                    
    
#korr_testit(nykykys,vasteet,faktorit,osajoukot,kynnys=0.01)
    
    

### liitu kouluympäristön tekijät esikäsittelyssä

In [21]:
liitu_lyh = {'Jokaiseen koulupäivään sisältyy vähintään 30 min yhtenäinen liikkumisvälitunti':'l_valitunti_30min',
       'Koululla on liikunnallista kerhotoimintaa':'l_kerhotoiminta',
       'Koululla on toimivat polkupyörien ja kypärien säilytysratkaisut':'l_pyora_sailytys',
       'Koulun liikuntasali on käytössä välituntiliikkumisessa':'l_valitunti_sali',
       'Koulun piha-aluetta ja lähiympäristöä hyödynnetään oppitunneilla':'l_oppitunti_pihaymp',
       'Koulun pihalla on pelialue ja riittävästi muita liikunnallisia toimintapaikkoja':'l_piha_toiminta',
       'Koulussa kannustetaan oppilaita liikkumaan koulumatkat kävellen tai pyöräillen':'l_koulumatkat',
       'Koulussa on käytössä koulupyöriä, joita voidaan lainata koulupäivän aikana':'l_pyora_lainaus',
       'Koulussa on riittävästi välituntivälineitä':'l_valitunti_valine',
       'Oppilaat ohjaavat muita oppilaita välituntiliikunnassa':'l_valitunti_opp_ohj',
       'Oppitunneilla tauotetaan istumista päivittäin':'l_oppitunti_tauot',
       'Välituntivälineet ovat kaikkien saatavilla':'l_valitunti_valineet'}
liitu['pisteet']=liitu.vast_K.fillna(0)-liitu.vast_E.fillna(0)
liitupts = liitu.reset_index().pivot(columns='Muuttuja',index='Koulu_ID',values='pisteet').rename(columns=liitu_lyh).fillna(0)
liitupts.sample(5).T

Koulu_ID,111,112,104,101,154
Muuttuja,,,,,
l_valitunti_30min,-16.2,43.4,-35.9,-31.5,69.9
l_kerhotoiminta,-87.8,-13.9,-58.0,-5.8,49.1
l_pyora_sailytys,0.0,16.6,50.0,11.4,21.6
l_valitunti_sali,69.6,-65.5,71.8,-57.2,-75.4
l_oppitunti_pihaymp,0.0,0.0,50.0,34.3,49.0
l_piha_toiminta,0.0,50.0,50.0,42.9,64.0
l_koulumatkat,-9.0,0.0,-52.9,-20.6,53.9
l_pyora_lainaus,0.0,-70.0,50.0,-68.5,-70.6
l_valitunti_valine,-67.6,0.0,-86.8,-5.7,32.1


### Edistävä toiminta - korrelaatio liikkumiseen ja suoriutumiseen

In [22]:
nykyindeksi = pd.read_csv("../data/nykytila_indeksit.csv")
nykyindeksi.set_index('Koulu_ID',inplace=True)
nykyindeksi = nykyindeksi.groupby('Koulu_ID').mean()
nykyindeksi.columns = ['k_'+c for c in nykyindeksi.columns]
nykyindeksi.sample(5)

,k_strategia,k_olosuhteet,k_toiminta,k_tuki
Koulu_ID,,,,
101,18.0,19.0,37.0,17.0
113,14.0,17.0,44.0,14.0
196,12.0,19.0,45.0,12.0
106,20.0,24.0,52.0,16.0
184,8.0,9.0,33.0,6.0


In [23]:
toimet = (ktk. #merge(perus,on='Koulu_ID').
    merge(nykyindeksi,on='Koulu_ID').
    merge(liitupts,on='Koulu_ID').
    merge(move,on=['Koulu_ID','sukupuoli','luokka'])
         )


print(toimet.shape)
toimet.sample(5).T

(56, 26)


,44,52,21,10,25
Koulu_ID,135,135,106,147,124
vuosi,2023,2023,2023,2023,2023
g_ex_012d,22.3,19.6,7.3,5.4,2.1
g_ex_34d,28.1,33.3,22.0,7.3,12.5
g_ex_56d,28.1,31.4,31.7,32.7,35.5
g_ex_7d,21.5,15.7,39.0,54.5,50.0
sukupuoli,t,p,p,t,p
luokka,8,8,5,5,5
k_strategia,17.0,17.0,20.0,14.0,20.0
k_olosuhteet,23.0,23.0,24.0,16.0,22.0


In [24]:
# toimet.to_excel("../output/toimet_2023.xlsx")

In [25]:
# osajoukot = [[],['luokka'],['sukupuoli'],['luokka','sukupuoli']]

faktorit = list(toimet.columns[(toimet.columns.str.startswith('k_')) | (toimet.columns.str.startswith('l_'))])
vasteet = ['osuus_A','osuus_C','g_ex_012d','g_ex_7d']
osajoukot = [[],['luokka'],['sukupuoli']]

korr_testit(toimet,vasteet,faktorit,osajoukot)#to_excel('../output/toimet_korr_2023.xlsx')

================= Ryhmä yht N=56 ===============
             osuus_A x l_pyora_sailytys     Pearson r -0.482 p=0.000 N=56 *****
             osuus_A x l_valitunti_sali     Pearson r +0.298 p=0.026 N=56 
             osuus_C x l_pyora_sailytys     Pearson r +0.436 p=0.001 N=56 ****
             osuus_C x l_valitunti_sali     Pearson r -0.272 p=0.043 N=56 
           g_ex_012d x l_pyora_sailytys     Pearson r -0.397 p=0.002 N=56 **
             g_ex_7d x k_strategia          Pearson r -0.307 p=0.021 N=56 
             g_ex_7d x l_valitunti_30min    Pearson r +0.316 p=0.018 N=56 *
             g_ex_7d x l_pyora_sailytys     Pearson r +0.288 p=0.031 N=56 
================= Ryhmä (5,) N=40 ===============
             osuus_A x l_pyora_sailytys     Pearson r -0.457 p=0.003 N=40 **
             osuus_C x l_pyora_sailytys     Pearson r +0.405 p=0.009 N=40 *
             osuus_C x l_valitunti_valineet Pearson r +0.324 p=0.042 N=40 
             g_ex_7d x k_strategia          Pearson r -0.358 

,osajoukko,vaste,tekija,r,p_arvo,otos_N
0,yht,osuus_A,l_pyora_sailytys,-0.481639,0.000171,56
1,yht,osuus_A,l_valitunti_sali,0.297684,0.025867,56
2,yht,osuus_C,l_pyora_sailytys,0.435957,0.000784,56
3,yht,osuus_C,l_valitunti_sali,-0.271745,0.042766,56
4,yht,g_ex_012d,l_pyora_sailytys,-0.397438,0.002421,56
5,yht,g_ex_7d,k_strategia,-0.306928,0.021398,56
6,yht,g_ex_7d,l_valitunti_30min,0.315945,0.017687,56
7,yht,g_ex_7d,l_pyora_sailytys,0.287949,0.031395,56
8,"(5,)",osuus_A,l_pyora_sailytys,-0.457135,0.003022,40
9,"(5,)",osuus_C,l_pyora_sailytys,0.405388,0.009457,40


## Korrelaatio move ja ktk datan välillä

In [26]:
movektk = movedf.merge(ktk,on=['Koulu_ID','luokka','sukupuoli'])
movektk.drop(columns=['laji', 'osuus_ref_A',
       'osuus_ref_B', 'osuus_B', 'osuus_ref_C', 
       'vuosi_y',  'g_ex_34d', 'g_ex_56d'], inplace=True)
movektk.set_index('Koulu_ID',inplace=True)
movektk.head(10)

,vuosi_x,luokka,sukupuoli,osuus_A,osuus_C,g_ex_012d,g_ex_7d
Koulu_ID,,,,,,,
101,2022,5,t,47.4,21.1,14.8,18.5
101,2022,5,p,56.5,26.1,0.0,60.0
101,2022,8,t,50.0,18.8,16.8,24.7
101,2022,8,p,33.3,57.1,10.0,34.3
102,2022,5,t,21.4,46.4,2.3,37.2
102,2022,5,p,24.3,35.1,2.8,31.5
102,2022,8,t,35.7,28.6,34.4,18.8
102,2022,8,p,8.7,60.9,2.7,40.5
103,2022,5,t,45.5,25.0,7.6,46.2


In [27]:


osajoukot = [[],['luokka'],['sukupuoli'],['luokka','sukupuoli']]
vasteet = ['osuus_A','osuus_C']
faktorit = ['g_ex_012d','g_ex_7d']
y=korr_testit(movektk,vasteet,faktorit,osajoukot,kynnys=.4)


================= Ryhmä yht N=190 ===============
             osuus_A x g_ex_012d            Pearson r +0.128 p=0.079 N=190 
             osuus_C x g_ex_012d            Pearson r -0.192 p=0.008 N=190 *
             osuus_C x g_ex_7d              Pearson r +0.136 p=0.062 N=190 
================= Ryhmä (5,) N=126 ===============
             osuus_A x g_ex_012d            Pearson r +0.186 p=0.037 N=126 
             osuus_A x g_ex_7d              Pearson r -0.106 p=0.238 N=126 
             osuus_C x g_ex_012d            Pearson r -0.259 p=0.003 N=126 **
             osuus_C x g_ex_7d              Pearson r +0.224 p=0.012 N=126 *
================= Ryhmä (8,) N=64 ===============
             osuus_A x g_ex_012d            Pearson r +0.401 p=0.001 N=64 ***
             osuus_A x g_ex_7d              Pearson r -0.274 p=0.028 N=64 
             osuus_C x g_ex_012d            Pearson r -0.399 p=0.001 N=64 ***
             osuus_C x g_ex_7d              Pearson r +0.347 p=0.005 N=64 **
=====

In [28]:
y #.to_excel("../output/move_ktk_korr_2023.xlsx")
movektk # .to_excel("../output/move_ktk_2023.xlsx")

,vuosi_x,luokka,sukupuoli,osuus_A,osuus_C,g_ex_012d,g_ex_7d
Koulu_ID,,,,,,,
101,2022,5,t,47.4,21.1,14.8,18.5
101,2022,5,p,56.5,26.1,0.0,60.0
101,2022,8,t,50.0,18.8,16.8,24.7
101,2022,8,p,33.3,57.1,10.0,34.3
102,2022,5,t,21.4,46.4,2.3,37.2
...,...,...,...,...,...,...,...
195,2022,5,p,30.0,50.0,8.7,34.8
196,2022,5,t,92.9,0.0,0.0,90.0
196,2022,5,p,100.0,0.0,4.8,52.4


### Tarkemmin nykytilan arvioista - korrelaatio per kysymys

In [29]:
nykykys = (ktk.merge(move,on=['Koulu_ID','sukupuoli','luokka']).merge(nyky_k,on='Koulu_ID')
    .drop(columns=['Vapaa sana'])).drop_duplicates()

print(nykykys.shape)
nykykys.sample(20).T
# nykykys.to_excel("../output/nykykys_2023.xlsx")

(124, 46)


,62,29,39,6,51,28,78,99,73,104,75,94,49,13,86,21,79,34,114,123
Koulu_ID,149,176,193,115,118,175,186,180,177,193,180,135,114,137,101,154,187,182,137,193
vuosi,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023
g_ex_012d,0.0,4.3,0.0,9.7,21.2,0.0,13.2,12.0,9.1,16.1,7.3,22.3,16.6,14.0,16.8,5.1,3.7,9.6,18.0,7.2
g_ex_34d,27.9,11.4,27.2,16.2,18.2,10.0,29.0,44.0,15.2,22.6,7.3,28.1,20.0,18.0,30.4,12.0,14.8,26.2,24.0,14.3
g_ex_56d,30.2,34.3,27.2,32.3,21.2,45.0,21.1,24.0,27.3,29.0,29.1,28.1,23.3,30.0,28.1,48.2,14.8,31.0,32.0,17.8
g_ex_7d,41.9,50.0,45.5,41.9,39.4,45.0,36.8,20.0,48.5,32.3,56.4,21.5,40.0,38.0,24.7,34.5,66.7,33.3,26.0,60.7
sukupuoli,p,t,t,t,p,t,p,t,p,t,p,t,p,t,t,t,p,t,p,p
luokka,5,5,5,5,5,5,5,8,5,8,5,8,5,5,8,5,5,5,8,8
osuus_A,30.0,52.8,11.5,62.0,27.5,32.1,68.6,61.5,22.2,14.3,58.6,46.0,36.0,61.4,50.0,29.7,45.3,13.0,35.5,14.6
osuus_C,40.0,33.3,65.4,8.0,42.5,32.1,11.4,15.4,66.7,57.1,13.8,28.6,24.0,13.6,18.8,48.6,30.2,56.5,32.3,63.4


In [30]:
#nk = nykykys.drop(columns=['sukupuoli','luokka']).groupby(['Koulu_ID']).mean()


In [31]:
# dia 1/2

faktorit = nykykys.columns[10:]


osajoukot = [[],['luokka'],['sukupuoli']]
vasteet = ['osuus_A','osuus_C','g_ex_012d','g_ex_7d']
x=korr_testit(nykykys,vasteet,faktorit,osajoukot,kynnys=.05)

================= Ryhmä yht N=124 ===============
             osuus_A x 1.6 Rehtori on sitou Pearson r +0.202 p=0.025 N=124 
             osuus_A x 3.1 Oppilaat osallis Pearson r -0.210 p=0.019 N=124 
             osuus_A x 5.1 Oppilaita kannus Pearson r -0.309 p=0.000 N=124 ****
             osuus_A x 5.3 Koululla on liik Pearson r +0.179 p=0.047 N=124 
             osuus_C x 1.6 Rehtori on sitou Pearson r -0.239 p=0.007 N=124 *
             osuus_C x 2.1 Opetuksessa on s Pearson r -0.252 p=0.005 N=124 **
             osuus_C x 3.1 Oppilaat osallis Pearson r +0.180 p=0.045 N=124 
             osuus_C x 5.1 Oppilaita kannus Pearson r +0.292 p=0.001 N=124 ***
           g_ex_012d x 1.3 Koulupäivän aika Pearson r -0.231 p=0.010 N=124 *
           g_ex_012d x 3.1 Oppilaat osallis Pearson r -0.190 p=0.035 N=124 
           g_ex_012d x 4.1 Välitunneilla me Pearson r -0.179 p=0.047 N=124 
             g_ex_7d x 1.5 Liikkuva koulu - Pearson r -0.274 p=0.002 N=124 ***
             g_ex_7d x 3

In [32]:
# dia 2/2: osaryhmä luokka+sukupuoli: 5p, 5t ..

faktorit = nykykys.columns[10:]


osajoukot = [['luokka','sukupuoli']]
vasteet = ['g_ex_012d','g_ex_7d']
y=korr_testit(nykykys,vasteet,faktorit,osajoukot,kynnys=.2)

================= Ryhmä (5, 'p') N=43 ===============
           g_ex_012d x 1.3 Koulupäivän aika Pearson r -0.319 p=0.037 N=43 
           g_ex_012d x 2.1 Opetuksessa on s Pearson r +0.206 p=0.184 N=43 
           g_ex_012d x 2.2 Toiminnallisia m Pearson r -0.214 p=0.167 N=43 
             g_ex_7d x 6.1 Opettajat tekevä Pearson r -0.223 p=0.151 N=43 
================= Ryhmä (5, 't') N=43 ===============
           g_ex_012d x 1.5 Liikkuva koulu - Pearson r +0.261 p=0.092 N=43 
           g_ex_012d x 2.1 Opetuksessa on s Pearson r +0.302 p=0.049 N=43 
           g_ex_012d x 2.4 Koulun opetustil Pearson r +0.325 p=0.033 N=43 
           g_ex_012d x 4.7 Koulun liikuntas Pearson r +0.269 p=0.081 N=43 
           g_ex_012d x 5.1 Oppilaita kannus Pearson r +0.261 p=0.091 N=43 
           g_ex_012d x 6.2 Huoltajien kanss Pearson r +0.404 p=0.007 N=43 *
             g_ex_7d x 1.4 Hyvinvoinnin ja  Pearson r -0.330 p=0.031 N=43 
             g_ex_7d x 1.5 Liikkuva koulu - Pearson r -0.568 p=0.0

In [33]:
y = pd.concat([df.sort_values('p_arvo').head(3) for (d,df) in y.groupby(['osajoukko'])]).sort_values('vaste')
#y.to_excel('../output/nykykys_korr_ktk_lk_sukup_2023.xlsx')
y

,osajoukko,vaste,tekija,r,p_arvo,otos_N
0,"(5, 'p')",g_ex_012d,1.3 Koulupäivän aikainen liikkuminen on kirjat...,-0.319017,0.037058,43
2,"(5, 'p')",g_ex_012d,2.2 Toiminnallisia menetelmiä hyödynnetään eri...,-0.214361,0.167480,43
9,"(5, 't')",g_ex_012d,6.2 Huoltajien kanssa tehdään yhteistyötä käve...,0.404064,0.007205,43
31,"(8, 't')",g_ex_012d,1.3 Koulupäivän aikainen liikkuminen on kirjat...,-0.357450,0.132976,19
3,"(5, 'p')",g_ex_7d,6.1 Opettajat tekevät keskenään yhteistyötä yl...,-0.222822,0.150942,43
11,"(5, 't')",g_ex_7d,1.5 Liikkuva koulu -toiminta on kirjattu kunta...,-0.568480,0.000070,43
18,"(5, 't')",g_ex_7d,6.3 Kouluterveydenhoitaja on mukana oppilaiden...,-0.359008,0.018068,43
28,"(8, 'p')",g_ex_7d,6.2 Huoltajien kanssa tehdään yhteistyötä käve...,0.485243,0.035210,19
26,"(8, 'p')",g_ex_7d,5.1 Oppilaita kannustetaan liikkumaan koulumat...,0.472456,0.041090,19
27,"(8, 'p')",g_ex_7d,6.1 Opettajat tekevät keskenään yhteistyötä yl...,0.466240,0.044207,19


### Olosuhteet ja muut tekijät - korrelaatio liikkumiseen ja suoriutumiseen

In [34]:
olo_muut = (ktk.merge(perus,on='Koulu_ID').
       merge(piha,on='Koulu_ID').
       merge(sali,on='Koulu_ID').
       merge(pyora,on='Koulu_ID').
           merge(move,on=['Koulu_ID','sukupuoli','luokka']))\
        .drop(columns=['stara_kommentti'])

olo_muut.shape

(169, 17)

In [35]:
olo_muut.sample(4).T

,142,96,48,12
Koulu_ID,103,176,188,120
vuosi,2023,2023,2023,2023
g_ex_012d,12.5,4.2,0.0,0.0
g_ex_34d,17.5,10.1,32.1,10.0
g_ex_56d,27.5,39.1,46.5,60.0
g_ex_7d,42.5,46.4,21.4,30.0
sukupuoli,p,p,t,t
luokka,8,5,5,5
ses,93.0,59.0,71.0,199.0
painot,0.0,0.0,0.0,0.0


In [36]:
# olo_muut.to_excel("../output/olo_muut_2023.xlsx")

In [37]:
## Korrelaatioiden kaivelu_
osajoukot = [['luokka','sukupuoli']]
faktorit = ['ses']
vasteet = ['osuus_C']

korr_testit(olo_muut,vasteet,faktorit,osajoukot,kynnys=1) #.to_excel('../output/olo_muut_korr_2023.xlsx')

================= Ryhmä (5, 'p') N=56 ===============
             osuus_C x ses                  Pearson r -0.273 p=0.044 N=55 
================= Ryhmä (5, 't') N=55 ===============
             osuus_C x ses                  Pearson r -0.304 p=0.025 N=54 
================= Ryhmä (8, 'p') N=29 ===============
             osuus_C x ses                  Pearson r -0.092 p=0.637 N=29 
================= Ryhmä (8, 't') N=29 ===============
             osuus_C x ses                  Pearson r -0.316 p=0.095 N=29 


,osajoukko,vaste,tekija,r,p_arvo,otos_N
0,"(5, 'p')",osuus_C,ses,-0.273029,0.043714,55
1,"(5, 't')",osuus_C,ses,-0.304337,0.025257,54
2,"(8, 'p')",osuus_C,ses,-0.091527,0.636785,29
3,"(8, 't')",osuus_C,ses,-0.316273,0.094627,29


In [38]:

osajoukot = [[],['luokka'],['sukupuoli']]
faktorit = ['pihakunto','ses','salim2','n_opp','pk_opp','painot']
vasteet = ['osuus_A','osuus_C','g_ex_012d','g_ex_7d']


korr_testit(olo_muut,vasteet,faktorit,osajoukot) #.to_excel('../output/olo_muut_korr_2023.xlsx')

================= Ryhmä yht N=169 ===============
             osuus_A x ses                  Pearson r +0.225 p=0.003 N=167 **
             osuus_A x painot               Pearson r -0.224 p=0.003 N=169 **
             osuus_C x ses                  Pearson r -0.245 p=0.001 N=167 ***
             osuus_C x painot               Pearson r +0.220 p=0.004 N=169 **
           g_ex_012d x ses                  Pearson r +0.267 p=0.000 N=167 ****
           g_ex_012d x salim2               Pearson r +0.254 p=0.001 N=169 ***
           g_ex_012d x n_opp                Pearson r +0.201 p=0.009 N=169 *
             g_ex_7d x ses                  Pearson r -0.216 p=0.005 N=167 **
             g_ex_7d x salim2               Pearson r -0.209 p=0.006 N=169 **
================= Ryhmä (5,) N=111 ===============
             osuus_A x ses                  Pearson r +0.267 p=0.005 N=109 **
             osuus_C x ses                  Pearson r -0.281 p=0.003 N=109 **
           g_ex_012d x ses            

,osajoukko,vaste,tekija,r,p_arvo,otos_N
0,yht,osuus_A,ses,0.225433,0.003399,167
1,yht,osuus_A,painot,-0.224093,0.003401,169
2,yht,osuus_C,ses,-0.244839,0.001428,167
3,yht,osuus_C,painot,0.220074,0.004040,169
4,yht,g_ex_012d,ses,0.266683,0.000494,167
5,yht,g_ex_012d,salim2,0.254128,0.000856,169
6,yht,g_ex_012d,n_opp,0.201260,0.008694,169
7,yht,g_ex_7d,ses,-0.215906,0.005074,167
8,yht,g_ex_7d,salim2,-0.208618,0.006490,169
9,"(5,)",osuus_A,ses,0.267040,0.005001,109


In [39]:
aa = pd.DataFrame(movedf.groupby(['sukupuoli','luokka']).osuus_A.mean().round(2))
aa['ref']=movedf.groupby(['sukupuoli','luokka']).osuus_ref_A.mean()
aa

osuus_A   ref
sukupuoli luokka               
p         5         38.26  41.1
          8         28.49  35.5
t         5         44.33  40.3
          8         36.62  33.2

In [40]:
cc = pd.DataFrame(movedf.groupby(['sukupuoli','luokka']).osuus_C.mean().round(2))
cc['ref']=movedf.groupby(['sukupuoli','luokka']).osuus_ref_C.mean()
cc

osuus_C   ref
sukupuoli luokka               
p         5         33.65  31.9
          8         42.62  35.6
t         5         29.01  32.1
          8         32.84  34.9

In [41]:
# KTK histogrammiryhmät

# ktk.to_excel("../output/ktk_2023_ryhmat_koulut.xlsx")
ktk

,Koulu_ID,vuosi,g_ex_012d,g_ex_34d,g_ex_56d,g_ex_7d,sukupuoli,luokka
0,101,2023,14.8,18.5,48.1,18.5,t,5
1,102,2023,2.3,16.3,44.2,37.2,t,5
2,103,2023,7.6,15.0,31.2,46.2,t,5
3,104,2023,2.0,4.1,24.5,69.4,t,5
4,106,2023,16.4,14.9,31.3,37.3,t,5
...,...,...,...,...,...,...,...,...
37,189,2023,9.5,26.0,30.1,34.2,p,8
38,193,2023,7.2,14.3,17.8,60.7,p,8
39,194,2023,11.8,30.8,32.3,25.0,p,8
40,200,2023,100.0,0.0,0.0,0.0,p,8


In [42]:
ddf=[]
for (g,df) in ktk.groupby(["sukupuoli","luokka"]):
    df2=pd.DataFrame((((df.g_ex_012d+5)/10).astype(int)*10).value_counts().reindex(np.arange(0,110,10),fill_value=0))
    df2['grp']=str(g)
    ddf.append(df2)

ktk012_histo=pd.concat(ddf).pivot(values='count',columns='grp')
# ktk012_histo.to_excel("../output/ktk_2023_012d_ryhmat.xlsx")
ktk012_histo

grp,"('p', 5)","('p', 8)","('t', 5)","('t', 8)"
g_ex_012d,,,,
0,40,6,41,1
10,43,26,43,9
20,5,8,4,21
30,2,1,1,3
40,0,0,0,5
50,0,0,1,0
60,0,0,0,0
70,0,0,0,1
80,0,0,0,1


In [43]:

ddf=[]
for (g,df) in ktk.groupby(["sukupuoli","luokka"]):
    df2=pd.DataFrame((((df.g_ex_7d+5)/10).astype(int)*10).value_counts().reindex(np.arange(0,110,10),fill_value=0))
    df2['grp']=str(g)
    ddf.append(df2)

ktk7_histo=pd.concat(ddf).pivot(values='count',columns='grp')
# ktk7_histo.to_excel("../output/ktk_2023_7d_ryhmat.xlsx")
ktk7_histo

grp,"('p', 5)","('p', 8)","('t', 5)","('t', 8)"
g_ex_7d,,,,
0,1,1,1,2
10,0,1,2,3
20,0,9,7,21
30,10,19,29,12
40,38,8,28,1
50,24,3,19,1
60,14,1,2,0
70,2,0,1,1
80,1,0,0,0


# Nykytilan osasummat

In [44]:
nykysum_df = pd.read_csv("../data/nykytila_indeksit.csv").groupby('Koulu_ID').mean()


In [45]:
nykysum_df.columns

Index(['strategia', 'olosuhteet', 'toiminta', 'tuki'], dtype='object')

In [46]:
nyky_olo_muut = nykysum_df.merge(nykykys,on='Koulu_ID').set_index('Koulu_ID')

In [47]:

osajoukot = [[],['luokka'],['sukupuoli'],['luokka','sukupuoli']]
faktorit = list(nykysum_df.columns)
vasteet = ['osuus_A','g_ex_012d','osuus_C','g_ex_7d']


korr_testit(nyky_olo_muut,vasteet,faktorit,osajoukot,kynnys=0.1) #.to_excel('../output/olo_muut_korr_2023.xlsx')

================= Ryhmä yht N=124 ===============
================= Ryhmä (5,) N=86 ===============
             g_ex_7d x strategia            Pearson r -0.273 p=0.011 N=86 *
================= Ryhmä (8,) N=38 ===============
================= Ryhmä ('p',) N=62 ===============
================= Ryhmä ('t',) N=62 ===============
================= Ryhmä (5, 'p') N=43 ===============
================= Ryhmä (5, 't') N=43 ===============
           g_ex_012d x olosuhteet           Pearson r +0.302 p=0.049 N=43 
             g_ex_7d x strategia            Pearson r -0.372 p=0.014 N=43 *
================= Ryhmä (8, 'p') N=19 ===============
             g_ex_7d x tuki                 Pearson r +0.452 p=0.052 N=19 
================= Ryhmä (8, 't') N=19 ===============


,osajoukko,vaste,tekija,r,p_arvo,otos_N
0,"(5,)",g_ex_7d,strategia,-0.272807,0.011044,86
1,"(5, 't')",g_ex_012d,olosuhteet,0.301790,0.049209,43
2,"(5, 't')",g_ex_7d,strategia,-0.372089,0.014011,43
3,"(8, 'p')",g_ex_7d,tuki,0.452394,0.051801,19


# Move! 20m H:gin keskiarvot

In [48]:
pd.DataFrame(movedf,columns=['Koulu_ID','luokka','sukupuoli']+[c for c in movedf.columns if 'osuus' in c]).\
  set_index('Koulu_ID').groupby(['luokka','sukupuoli']).mean().round(2)

osuus_ref_A  osuus_A  osuus_ref_B  osuus_B  osuus_ref_C  \
luokka sukupuoli                                                            
5      p                 41.1    38.26         27.0    28.10         31.9   
       t                 40.3    44.33         27.6    26.64         32.1   
8      p                 35.5    28.49         28.9    28.90         35.6   
       t                 33.2    36.62         31.9    30.54         34.9   

                  osuus_C  
luokka sukupuoli           
5      p            33.65  
       t            29.01  
8      p            42.62  
       t            32.84

# N:t


In [49]:
len(perus.index.unique()), perus.index.name # Koulujen perustiedot

(102, 'Koulu_ID')

In [50]:
len(nyky.index.unique()), nyky.index.name # Peruskoulujen nykytilan arvio

(74, 'Koulu_ID')

In [51]:
len(ktk.Koulu_ID.unique()) # Kouluterveyskysely, liikkuvuus

97

In [52]:
len(movedf.Koulu_ID.unique()) # Move! mittaukset

71

In [53]:
len(liitu.index.unique()), liitu.index.name # LIITU-kysely

(32, 'Koulu_ID')

In [54]:
len(piha.index.unique()), piha.index.name # Koulupihojen kuntoarviot

(94, 'Koulu_ID')

In [55]:
len(pyora.index.unique()), pyora.index.name # Pyöräpaikkojen lkm

(99, 'Koulu_ID')

In [56]:
len(sali.index.unique()),sali.index.name # Koulujen salit

(88, 'Koulu_ID')

In [57]:
sum(perus.ses>0) # SES-indeksit laskettu n:lle koululle... (joukossa tyhjiä)

99

In [58]:
lu.nykytila_df.Vastaamisajankohta.value_counts().sort_index()

Vastaamisajankohta
2018 syksy     1
2019 kevät    92
2019 syksy     9
2020 kevät    15
2020 syksy    12
2021 Syksy    21
2021 kevät    12
2022 kevät     1
2022 syksy    43
2023 kevät    32
Name: count, dtype: int64

# Toimintasuunnitelma luokat

In [59]:
toims_df = pd.read_excel("../input/Koulujen_toimintasuunnitelmat_suojattu_2022-2023.xlsx")

In [60]:
toim = pd.DataFrame(toims_df,columns=toims_df.columns[2:]).set_index(toims_df.Koulu_ID).dropna()
toimcols = {'Toiminnan organisoiminen':'toiminnan_org', 'Oppilaiden osallisuus':'opp_osallisuus', 
            'Koulumatkat':'koulumatkat','Välitunnit':'valitunnit', 'Koulun pihat, tilat ja ympäristö':'pihat_ymp', 
            'Kerhotoiminta':'kerhotoiminta','Henkilökunnan osallistuminen, osaaminen ja hyvinvointi':'henkilokunta',
            'Oppitunnit, opetuskäytännöt ja oppimisympäristöt':'opp_tunnit_kayt_ymp',
            'Yhteistyö oppilaiden liikkumisen edistämiseksi':'yhteistyo'}
toim.rename(columns=toimcols,inplace=True)
toim

,toiminnan_org,opp_osallisuus,koulumatkat,valitunnit,pihat_ymp,kerhotoiminta,henkilokunta,opp_tunnit_kayt_ymp,yhteistyo
Koulu_ID,,,,,,,,,
101,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
102,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
103,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
189,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...
184,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
185,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
186,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0


In [61]:
ktk_toim = ktk.merge(toim,on='Koulu_ID')

In [62]:
ktk_toimintasuun_korr = korr_testit(ktk_toim,['g_ex_7d','g_ex_012d'],ktk_toim.columns[8:],[['luokka'],['sukupuoli']])
ktk_toimintasuun_korr

================= Ryhmä (5,) N=158 ===============
             g_ex_7d x opp_tunnit_kayt_ymp  Pearson r -0.185 p=0.020 N=158 
================= Ryhmä (8,) N=76 ===============
             g_ex_7d x opp_tunnit_kayt_ymp  Pearson r +0.283 p=0.013 N=76 *
================= Ryhmä ('p',) N=117 ===============
================= Ryhmä ('t',) N=117 ===============
             g_ex_7d x valitunnit           Pearson r +0.197 p=0.033 N=117 
           g_ex_012d x pihat_ymp            Pearson r +0.188 p=0.042 N=117 
           g_ex_012d x kerhotoiminta        Pearson r +0.187 p=0.043 N=117 


,osajoukko,vaste,tekija,r,p_arvo,otos_N
0,"(5,)",g_ex_7d,opp_tunnit_kayt_ymp,-0.185144,0.019864,158
1,"(8,)",g_ex_7d,opp_tunnit_kayt_ymp,0.283059,0.013225,76
2,"('t',)",g_ex_7d,valitunnit,0.197264,0.033022,117
3,"('t',)",g_ex_012d,pihat_ymp,0.188072,0.042292,117
4,"('t',)",g_ex_012d,kerhotoiminta,0.187152,0.043330,117


In [63]:
toim.describe()

,toiminnan_org,opp_osallisuus,koulumatkat,valitunnit,pihat_ymp,kerhotoiminta,henkilokunta,opp_tunnit_kayt_ymp,yhteistyo
count,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000
mean,0.956044,0.351648,0.098901,0.670330,0.362637,0.197802,0.164835,0.406593,0.296703
std,0.206133,0.480130,0.300183,0.472698,0.483425,0.400549,0.373087,0.493919,0.459335
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [64]:
toim_ka = pd.DataFrame(list(toim.mean()),index=list(toimcols.keys()))
toim_ka

,0
Toiminnan organisoiminen,0.956044
Oppilaiden osallisuus,0.351648
Koulumatkat,0.098901
Välitunnit,0.670330
"Koulun pihat, tilat ja ympäristö",0.362637
Kerhotoiminta,0.197802
"Henkilökunnan osallistuminen, osaaminen ja hyvinvointi",0.164835
"Oppitunnit, opetuskäytännöt ja oppimisympäristöt",0.406593
Yhteistyö oppilaiden liikkumisen edistämiseksi,0.296703


In [65]:
#ktk_toimintasuun_korr.to_excel("../output/ktk_toimintas_korr_2023.xlsx")
#ktk_toim.to_excel("../output/ktk_toimintas_2023.xlsx")
#toim_ka.to_excel("../output/toimintas_ka_2023.xlsx")

## All inclusive datasheetit

In [66]:
kaikki1 = perus.merge(nyky,on='Koulu_ID',how='outer')\
                .merge(piha,on='Koulu_ID',how='outer')\
                .merge(pyora,on='Koulu_ID',how='outer')\
                .merge(sali,on='Koulu_ID',how='outer')\
                .merge(liitupts,on='Koulu_ID',how='outer')\
                .merge(sali,on='Koulu_ID',how='outer')\
                .merge(toim,on='Koulu_ID',how='outer')


kaikki2 = perus.merge(move,on=['Koulu_ID'],how='outer')\
                .merge(ktk,on=['Koulu_ID','luokka','sukupuoli'])

kaikki1.shape, kaikki2.shape

((102, 67), (190, 13))

In [67]:
kaikki1.columns

Index(['ses', 'painot', 'n_opp', 'k_1_1', 'k_1_2', 'k_1_3', 'k_1_4', 'k_1_5',
       'k_1_6', 'k_1_7', 'k_2_1', 'k_2_2', 'k_2_3', 'k_2_4', 'k_2_5', 'k_3_1',
       'k_3_2', 'k_3_3', 'k_3_4', 'k_3_5', 'k_3_6', 'k_3_7', 'k_3_8', 'k_4_1',
       'k_4_2', 'k_4_3', 'k_4_4', 'k_4_5', 'k_4_6', 'k_4_7', 'k_5_1', 'k_5_2',
       'k_5_3', 'k_5_4', 'k_6_1', 'k_6_2', 'k_6_3', 'k_6_4', 'k_6_5',
       'Vapaa sana', 'pihakunto', 'stara_kommentti', 'paikat', 'pk_opp',
       'salim2_x', 'l_valitunti_30min', 'l_kerhotoiminta', 'l_pyora_sailytys',
       'l_valitunti_sali', 'l_oppitunti_pihaymp', 'l_piha_toiminta',
       'l_koulumatkat', 'l_pyora_lainaus', 'l_valitunti_valine',
       'l_valitunti_opp_ohj', 'l_oppitunti_tauot', 'l_valitunti_valineet',
       'salim2_y', 'toiminnan_org', 'opp_osallisuus', 'koulumatkat',
       'valitunnit', 'pihat_ymp', 'kerhotoiminta', 'henkilokunta',
       'opp_tunnit_kayt_ymp', 'yhteistyo'],
      dtype='object')

In [68]:
kaikki1.sample(5)

,ses,painot,n_opp,k_1_1,k_1_2,k_1_3,k_1_4,k_1_5,k_1_6,k_1_7,...,salim2_y,toiminnan_org,opp_osallisuus,koulumatkat,valitunnit,pihat_ymp,kerhotoiminta,henkilokunta,opp_tunnit_kayt_ymp,yhteistyo
Koulu_ID,,,,,,,,,,,,,,,,,,,,,
102,62.0,0.0,872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,535.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
108,93.0,0.0,799,3.0,3.0,4.0,4.0,4.0,4.0,0.0,...,85.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
130,137.0,0.0,625,3.0,2.0,4.0,4.0,2.0,4.0,2.0,...,433.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
200,67.0,0.0,154,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,191.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163,123.0,0.0,415,3.0,3.0,2.0,4.0,3.0,3.0,1.0,...,221.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [69]:
kaikki2.sample(5)

,Koulu_ID,ses,painot,n_opp,luokka,sukupuoli,osuus_A,osuus_C,vuosi,g_ex_012d,g_ex_34d,g_ex_56d,g_ex_7d
24,108,93.0,0.0,799,8.0,p,30.8,30.8,2023,10.9,32.5,25.7,31.1
93,148,82.0,1.0,720,8.0,t,17.2,58.6,2023,18.1,13.6,22.7,45.5
120,160,124.0,1.0,1050,5.0,p,36.2,53.2,2023,2.8,11.2,33.8,52.1
21,108,93.0,0.0,799,5.0,t,69.2,11.5,2023,7.3,41.4,26.8,24.4
78,138,45.0,0.0,363,5.0,p,8.3,33.3,2023,8.0,10.0,24.0,58.0


In [70]:
kaikki1.to_excel("../output/muuttujat_2023_kouluittain.xlsx")
kaikki2.to_excel("../output/muuttujat_2023_ryhmittain.xlsx")